# The Critic Agent: Enhancing Multi-Agent Systems Through Constructive Evaluation

*By Rany ElHousieny*

*Last updated: May 17, 2025*

## Introduction

In this notebook, we'll explore the Critic agent, a crucial component of multi-agent AI systems. The Critic agent evaluates, challenges, and refines the work of other agents to ensure accuracy, completeness, and objectivity. We'll implement a Critic agent using LangChain and GPT-4.1, and demonstrate how it enhances the quality of AI-generated content.

## Setting Up the Environment

First, let's install the necessary packages and set up our environment:

In [5]:
# Install required packages
!pip install langchain-openai python-dotenv openai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [6]:
# Import necessary libraries
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()  # This loads the OPENAI_API_KEY from your .env file

True

In [7]:
# Import LangChain components
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Verify that we can connect to the OpenAI API
llm = ChatOpenAI(
    temperature=0.7, 
    model_name="gpt-4.1",  # Using GPT-4.1 as per user preference
    api_key=os.environ.get("OPENAI_API_KEY")
)

# Test the connection with a simple query
llm.invoke("Hello, are you ready to learn about the Critic agent?")

AIMessage(content="Hello! Yes, I'm ready and eager to learn about the Critic agent. Please go ahead and share what you know, or let me know how I can assist in learning or discussing the Critic agent.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 20, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_beec22d258', 'finish_reason': 'stop', 'logprobs': None}, id='run--fa1ee707-0ff5-43cd-99d0-2cb901dd0875-0', usage_metadata={'input_tokens': 20, 'output_tokens': 42, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## The Role of the Critic Agent

The Critic agent serves as a quality control mechanism within multi-agent systems. While other agents (like Researchers or Writers) focus on generating content, the Critic evaluates this content through a critical lens. This evaluation helps:

1. **Identify gaps and inconsistencies** in information
2. **Challenge assumptions** that might introduce bias
3. **Suggest improvements** to strengthen the overall quality
4. **Ensure balanced perspectives** by highlighting missing viewpoints
5. **Improve logical coherence** by questioning the reasoning process

Let's define the system prompt that establishes the Critic's role and responsibilities:

In [8]:
# Define the Critic Agent's system prompt
CRITIC_SYSTEM_PROMPT = """
You are a Critic Agent, part of a collaborative research assistant system. Your role is to evaluate 
and challenge information provided by the Researcher Agent to ensure accuracy, completeness, and objectivity.

Your responsibilities include:
1. Analyzing research findings for accuracy, completeness, and potential biases
2. Identifying gaps in the information or logical inconsistencies
3. Asking important questions that might have been overlooked
4. Suggesting improvements or alternative perspectives
5. Ensuring that the final information is balanced and well-rounded

Be constructive in your criticism. Your goal is not to dismiss the researcher's work, but to strengthen it.
Format your feedback in a clear, organized manner, highlighting specific points that need attention.

Remember, your ultimate goal is to ensure that the final research output is of the highest quality possible.
"""

# Let's also define a Researcher Agent's system prompt for our demonstration
RESEARCHER_SYSTEM_PROMPT = """
You are a Researcher Agent, part of a collaborative research assistant system. Your role is to gather and provide 
accurate, relevant information on any topic or question presented to you.

Your responsibilities include:
1. Analyzing research questions to understand what information is being requested
2. Providing comprehensive, well-structured responses based on your knowledge
3. Highlighting key points and important information
4. Being honest about the limitations of your knowledge
5. Maintaining objectivity and avoiding bias in your responses

Format your responses in a clear, organized manner with sections, bullet points, or numbered lists as appropriate.
Always cite your sources of information when possible.

Remember, your goal is to provide the most helpful, accurate information possible to assist in the research process.
"""

## Implementing the Critic Agent

Now, let's implement the Critic agent function. This function will:

1. Take a list of messages as input (including the original query and any previous agent responses)
2. Prepend the Critic's system prompt to establish its role
3. Create a ChatOpenAI instance with appropriate parameters
4. Send the messages to the language model and get a response
5. Return the Critic's evaluation

In [9]:
def critic_agent(messages):
    """Critic agent that evaluates and challenges the researcher's findings."""
    # Add the system prompt to guide the critic agent's behavior
    critic_messages = [SystemMessage(content=CRITIC_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.7,  # Slightly higher temperature for more creative criticism
        model_name="gpt-4.1",  # Using GPT-4.1 for advanced analytical capabilities
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model
    response = llm.invoke(critic_messages)
    
    # Print a preview of the response for monitoring
    print("\n--- Critic Agent Response ---")
    print(f"{response.content[:300]}...\n")
    
    return response

### Code Explanation

Let's break down each line of the `critic_agent` function to understand how it works:

```python
def critic_agent(messages):
```
- This defines a function named `critic_agent` that takes a list of `messages` as input. These messages will include the original query and any previous agent responses.

```python
critic_messages = [SystemMessage(content=CRITIC_SYSTEM_PROMPT)] + messages
```
- This creates a new list called `critic_messages` by prepending the Critic's system prompt to the existing messages.
- The `SystemMessage` class is used to format the prompt as a system message, which tells the language model what role to assume.
- This is crucial for establishing the Critic's role and responsibilities before it processes the other messages.

```python
llm = ChatOpenAI(
    temperature=0.7,  # Slightly higher temperature for more creative criticism
    model_name="gpt-4.1",  # Using GPT-4.1 for advanced analytical capabilities
    api_key=os.environ.get("OPENAI_API_KEY")
)
```
- This creates a `ChatOpenAI` instance, which is the interface to the OpenAI API.
- The `temperature` parameter is set to 0.7, which is slightly higher than what might be used for a fact-focused Researcher agent. This allows the Critic to be more creative in identifying potential issues and suggesting alternative approaches.
- The `model_name` parameter specifies which language model to use. We're using GPT-4.1 for its advanced analytical capabilities.
- The `api_key` parameter is set to the OpenAI API key retrieved from environment variables, which is a secure way to handle API keys.

```python
response = llm.invoke(critic_messages)
```
- This sends the prepared messages to the language model using the `invoke` method and gets a response.
- The response will contain the Critic's evaluation of the information provided by previous agents.

```python
print("\n--- Critic Agent Response ---")
print(f"{response.content[:300]}...\n")
```
- These lines print a preview of the Critic's response for monitoring purposes.
- Only the first 300 characters are shown to avoid cluttering the output.

```python
return response
```
- This returns the full response from the Critic agent, which can then be used by subsequent agents or presented to the user.

## Implementing the Researcher Agent

For our demonstration, we'll also need to implement a Researcher agent that will generate the initial content for the Critic to evaluate:

In [10]:
def researcher_agent(messages):
    """Researcher agent that analyzes questions and provides informative responses."""
    # Add the system prompt to guide the researcher agent's behavior
    researcher_messages = [SystemMessage(content=RESEARCHER_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.5,  # Lower temperature for more factual responses
        model_name="gpt-4.1",  # Using GPT-4.1 as per user preference
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model
    response = llm.invoke(researcher_messages)
    
    # Print a preview of the response for monitoring
    print("\n--- Researcher Agent Response ---")
    print(f"{response.content[:300]}...\n")
    
    return response

### Key Differences Between Researcher and Critic Agents

Notice the key differences between the Researcher and Critic agents:

1. **System Prompts**: Each agent has a distinct system prompt that defines its role and responsibilities.

2. **Temperature Settings**: 
   - Researcher: 0.5 (lower) - For more factual, focused responses
   - Critic: 0.7 (higher) - For more creative criticism and alternative perspectives

These differences reflect the distinct purposes of each agent in the multi-agent system.

## Creating a Simple Multi-Agent Pipeline

Now, let's create a simple sequential pipeline that combines the Researcher and Critic agents:

In [11]:
def run_multi_agent_system(question):
    """Run the multi-agent system with the given question."""
    print(f"\n\n{'='*50}")
    print(f"QUERY: {question}")
    print(f"{'='*50}\n")
    
    # Initialize the messages with the question
    messages = [HumanMessage(content=question)]
    
    # Run the researcher agent
    researcher_response = researcher_agent(messages)
    messages.append(researcher_response)
    
    # Run the critic agent
    critic_response = critic_agent(messages)
    messages.append(critic_response)
    
    return messages

### Pipeline Explanation

Let's break down how this pipeline works:

1. The function takes a question as input and initializes a list of messages with this question.

2. The Researcher agent processes the question and generates an initial response, which is added to the messages list.

3. The Critic agent receives both the original question and the Researcher's response, evaluates the information, and provides feedback.

4. The function returns the complete conversation history, which includes the original question, the Researcher's response, and the Critic's evaluation.

This sequential approach ensures that each agent has access to all relevant information from previous steps, allowing for a progressive refinement of the output.

## Testing the Multi-Agent System

Let's test our multi-agent system with a research question about quantum computing:

In [12]:
# Run the multi-agent system with a research question
result = run_multi_agent_system("What are the current challenges and future prospects of quantum computing?")



QUERY: What are the current challenges and future prospects of quantum computing?


--- Researcher Agent Response ---
**Current Challenges and Future Prospects of Quantum Computing**

---

## 1. Current Challenges

### a. Hardware Limitations
- **Qubit Quality:** Most quantum computers use noisy, error-prone qubits (quantum bits). Maintaining coherence (the ability to preserve quantum states) is difficult due to en...


--- Critic Agent Response ---
Your summary is comprehensive and covers many crucial aspects, but there are areas where the analysis could be deepened or clarified to ensure a more balanced and thorough perspective. Here is my critique and suggestions for improvement:

---

**1. Hardware Limitations:**
- **Missing Details on Comp...



In [13]:
# Display the full conversation
print("\n\nFull conversation:")
for i, message in enumerate(result):
    if isinstance(message, HumanMessage):
        print(f"\nHuman: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"\nAI: {message.content}")



Full conversation:

Human: What are the current challenges and future prospects of quantum computing?

AI: **Current Challenges and Future Prospects of Quantum Computing**

---

## 1. Current Challenges

### a. Hardware Limitations
- **Qubit Quality:** Most quantum computers use noisy, error-prone qubits (quantum bits). Maintaining coherence (the ability to preserve quantum states) is difficult due to environmental interference.
- **Scalability:** Building systems with more qubits while maintaining low error rates is a significant engineering challenge.
- **Error Correction:** Quantum error correction requires many physical qubits to create a single logical qubit, increasing hardware demands.

### b. Software and Algorithms
- **Algorithm Development:** Few quantum algorithms currently outperform classical algorithms for practical problems. Many quantum algorithms are still in the theoretical stage.
- **Programming Tools:** Quantum programming languages and frameworks are still maturi

## Analyzing the Critic's Response

Let's analyze how the Critic agent evaluates the Researcher's response:

1. **Identification of Gaps**: The Critic identifies areas where the Researcher's response may be incomplete or lacking depth.

2. **Constructive Feedback**: The Critic provides specific suggestions for improvement rather than simply pointing out flaws.

3. **Alternative Perspectives**: The Critic suggests alternative viewpoints or considerations that might have been overlooked.

4. **Balanced Evaluation**: The Critic acknowledges the strengths of the Researcher's response while also highlighting areas for improvement.

5. **Structured Format**: The Critic organizes its feedback in a clear, structured format that makes it easy to understand and act upon.

## Benefits of Including a Critic Agent

Including a dedicated Critic agent in a multi-agent system offers several advantages:

1. **Enhanced Quality Control**: The Critic provides a systematic evaluation step, reducing the likelihood of errors, biases, or incomplete information in the final output.

2. **Balanced Perspectives**: By actively seeking alternative viewpoints and identifying potential biases, the Critic helps ensure a more balanced and objective response.

3. **Depth and Comprehensiveness**: The Critic identifies gaps in information, prompting a more thorough exploration of the topic.

4. **Improved Reasoning**: By challenging assumptions and logical inconsistencies, the Critic contributes to more robust reasoning processes.

5. **Iterative Refinement**: The Critic's feedback can guide subsequent agents (or iterations) toward a progressively improved output.

## Advanced Implementation: Adding a Writer Agent

In a more complete multi-agent system, we might add a Writer agent that synthesizes the Researcher's information and the Critic's feedback into a final, refined response. Let's implement this:

In [14]:
# Define the Writer Agent's system prompt
WRITER_SYSTEM_PROMPT = """
You are a Writer Agent, part of a collaborative research assistant system. Your role is to synthesize 
information from the Researcher Agent and feedback from the Critic Agent into a coherent, well-written response.

Your responsibilities include:
1. Analyzing the information provided by the researcher and the feedback from the critic
2. Organizing the information in a logical, easy-to-understand structure
3. Presenting the information in a clear, engaging writing style
4. Balancing different perspectives and ensuring objectivity
5. Creating a final response that is comprehensive, accurate, and well-written

Format your response in a clear, organized manner with appropriate headings, paragraphs, and bullet points.
Use simple language to explain complex concepts, and provide examples where helpful.

Remember, your goal is to create a final response that effectively communicates the information to the user.
"""

def writer_agent(messages):
    """Writer agent that synthesizes information from the researcher and critic."""
    # Add the system prompt to guide the writer agent's behavior
    writer_messages = [SystemMessage(content=WRITER_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.6,  # Balanced temperature for creativity and accuracy
        model_name="gpt-4.1",  # Using GPT-4.1 as per user preference
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model
    response = llm.invoke(writer_messages)
    
    # Print a preview of the response for monitoring
    print("\n--- Writer Agent Response ---")
    print(f"{response.content[:300]}...\n")
    
    return response

In [15]:
def run_complete_multi_agent_system(question):
    """Run the complete multi-agent system with the given question."""
    print(f"\n\n{'='*50}")
    print(f"QUERY: {question}")
    print(f"{'='*50}\n")
    
    # Initialize the messages with the question
    messages = [HumanMessage(content=question)]
    
    # Run the researcher agent
    researcher_response = researcher_agent(messages)
    messages.append(researcher_response)
    
    # Run the critic agent
    critic_response = critic_agent(messages)
    messages.append(critic_response)
    
    # Run the writer agent
    writer_response = writer_agent(messages)
    messages.append(writer_response)
    
    # Return the final response
    return messages

In [16]:
# Run the complete multi-agent system with a research question
complete_result = run_complete_multi_agent_system("What are the ethical implications of artificial intelligence in healthcare?")



QUERY: What are the ethical implications of artificial intelligence in healthcare?


--- Researcher Agent Response ---
**Ethical Implications of Artificial Intelligence (AI) in Healthcare**

Artificial intelligence is rapidly transforming healthcare, offering significant benefits but also raising complex ethical challenges. Below is a structured overview of the key ethical implications:

---

### 1. **Patient Privac...


--- Critic Agent Response ---
Your summary of the ethical implications of AI in healthcare is thorough and well-cited. However, to further enhance the analysis for depth and balance, consider the following points:

---

**1. Completeness and Scope**

- **Beneficence and Nonmaleficence:** While you mention the risks and challenge...


--- Writer Agent Response ---
Ethical Implications of Artificial Intelligence in Healthcare

Artificial intelligence (AI) is rapidly changing healthcare, offering new opportunities for diagnosis, treatment, and patient care. However, its

In [17]:
# Display the final response from the Writer agent
print("\n\nFINAL RESPONSE:")
print(complete_result[-1].content)



FINAL RESPONSE:
Ethical Implications of Artificial Intelligence in Healthcare

Artificial intelligence (AI) is rapidly changing healthcare, offering new opportunities for diagnosis, treatment, and patient care. However, its use also raises important ethical questions. Below, we outline the main ethical implications, balancing both the potential benefits and the challenges.

---

1. Patient Privacy and Data Security

- **Sensitive Data Use:** AI systems often require large amounts of health data, which can include sensitive personal information. Protecting this data from unauthorized access and misuse is essential.
- **Informed Consent:** Patients may not fully understand how their data is used, especially when it’s shared for AI development. Clear and transparent consent processes are needed.

*Example:* If a hospital uses patient data to train an AI tool without explicit consent, it risks violating privacy laws and patient trust.

---

2. Bias, Fairness, and Health Equity

- **Algor

## The Value of the Critic in the Complete Pipeline

In this complete pipeline, the Critic agent serves as a crucial intermediary between the Researcher and Writer agents. The Critic:

1. Evaluates the Researcher's information for accuracy, completeness, and objectivity
2. Identifies gaps, inconsistencies, and potential biases
3. Suggests improvements and alternative perspectives
4. Provides structured feedback that the Writer can use to create a more refined final response

Without the Critic, the Writer would simply be reformatting the Researcher's information without the benefit of critical evaluation. The Critic ensures that the final response is not only well-written but also accurate, comprehensive, and balanced.

## Challenges and Considerations

While the Critic agent adds significant value to multi-agent systems, several challenges and considerations should be kept in mind:

1. **Balancing Criticism**: The Critic must provide substantive feedback without being overly negative or nitpicky. The system prompt plays a crucial role in striking this balance.

2. **Computational Overhead**: Adding a Critic agent increases the computational resources required for the system, as it involves an additional language model call.

3. **Potential for Contradictions**: In some cases, the Critic might contradict valid information provided by the Researcher, requiring careful handling by subsequent agents.

4. **Domain Expertise**: The Critic's effectiveness depends on its ability to evaluate specialized information, which may vary across domains.

5. **Feedback Integration**: The system must have a mechanism for integrating the Critic's feedback, either through additional agents or iterative processes.

## Conclusion

The Critic agent represents a powerful addition to multi-agent AI systems, embodying the principle that high-quality outputs often emerge from constructive criticism and iterative refinement. By systematically evaluating information for accuracy, completeness, and objectivity, the Critic helps ensure that the final output meets high standards of quality and reliability.

In this notebook, we've explored:

1. The role and responsibilities of the Critic agent
2. How to implement a Critic agent using LangChain and GPT-4.1
3. How the Critic fits into a multi-agent workflow
4. The benefits and challenges of including a Critic agent

As multi-agent systems continue to evolve, the role of the Critic is likely to become increasingly sophisticated, potentially incorporating specialized domain knowledge, learning from past evaluations, and adapting its criticism style based on the specific context and needs of the user.